<a href="https://colab.research.google.com/github/mishra-aanchal/cycleGAN-implementation/blob/main/cycleGAN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!wget https://newsmeter.in/wp-content/uploads/2020/06/New-Project-25.jpg
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz


--2021-03-15 08:54:27--  https://newsmeter.in/wp-content/uploads/2020/06/New-Project-25.jpg
Resolving newsmeter.in (newsmeter.in)... 104.21.79.201, 172.67.147.211, 2606:4700:3031::6815:4fc9, ...
Connecting to newsmeter.in (newsmeter.in)|104.21.79.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31584 (31K) [image/jpeg]
Saving to: ‘New-Project-25.jpg.2’

New-Project-25.jpg. 100%[===================>]  30.84K  --.-KB/s    in 0.006s  

2021-03-15 08:54:27 (5.09 MB/s) - ‘New-Project-25.jpg.2’ saved [31584/31584]



In [11]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import scipy.ndimage.interpolation

In [12]:
#Training PArams
learning_rate = 0.0002
batch_size = 32
epochs = 100000

#Network params
image_dimension = 784 #img sz is 28x28
#Discriminator Nodes
H_dim = 128

def xavier_init(shape):
  return tf.random_normal(shape = shape, stddev= 1./tf.sqrt(shape[0]/2.0))

In [13]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [14]:
#define placeholders for external input

X_A = tf.placeholder(tf.float32, shape = [None, image_dimension])
X_B = tf.placeholder(tf.float32, shape = [None, image_dimension])

In [15]:
# Define weights and biases for dictionaries for Discriminator A

Disc_A_W = { "disc_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "disc_final": tf.Variable(xavier_init([H_dim, 1]))}

Disc_A_Bias = { "disc_H" : tf.Variable(xavier_init([H_dim])),
             "disc_final": tf.Variable(xavier_init([1]))}


# Define weights and biases for dictionaries for Discriminator B

Disc_B_W = { "disc_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "disc_final": tf.Variable(xavier_init([H_dim, 1]))}

Disc_B_Bias = { "disc_H" : tf.Variable(xavier_init([H_dim])),
             "disc_final": tf.Variable(xavier_init([1]))}

# Define weights and biases for dictionaries for Generator transforming A to B

Gen_AB_W = { "Gen_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "Gen_final": tf.Variable(xavier_init([H_dim, image_dimension]))} #784 due to output dimension of the image

Gen_AB_Bias = { "Gen_H" : tf.Variable(xavier_init([H_dim])),
             "Gen_final": tf.Variable(xavier_init([image_dimension]))}


# Define weights and biases for dictionaries for Generator transforming B to A
Gen_BA_W = { "Gen_H" : tf.Variable(xavier_init([image_dimension, H_dim])),
             "Gen_final": tf.Variable(xavier_init([H_dim, image_dimension]))}

Gen_BA_Bias = { "Gen_H" : tf.Variable(xavier_init([H_dim])),
             "Gen_final": tf.Variable(xavier_init([image_dimension]))}

In [16]:
def Disc_A(x):
  D_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Disc_A_W["disc_H"]), Disc_A_Bias["disc_H"]))
  disc_output = (tf.add(tf.matmul(D_hidden_layer, Disc_A_W["disc_final"]), Disc_A_Bias["disc_final"]))
  disc_prob_output = tf.nn.sigmoid(disc_output)
  return disc_prob_output

def Disc_B(x):
  D_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Disc_B_W["disc_H"]), Disc_B_Bias["disc_H"]))
  disc_output = (tf.add(tf.matmul(D_hidden_layer, Disc_B_W["disc_final"]), Disc_B_Bias["disc_final"]))
  disc_prob_output = tf.nn.sigmoid(disc_output)
  return disc_prob_output

In [17]:
#Generator NW
def GenAB(x):
  G_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Gen_AB_W["Gen_H"]), Gen_AB_Bias["Gen_H"]))
  Gen_output = (tf.add(tf.matmul(G_hidden_layer, Gen_AB_W["Gen_final"]), Gen_AB_Bias["Gen_final"]))
  Gen_prob_output = tf.nn.sigmoid(Gen_output)
  return Gen_prob_output

def GenBA(x):
  G_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Gen_BA_W["Gen_H"]), Gen_BA_Bias["Gen_H"]))
  Gen_output = (tf.add(tf.matmul(G_hidden_layer, Gen_BA_W["Gen_final"]), Gen_BA_Bias["Gen_final"]))
  Gen_prob_output = tf.nn.sigmoid(Gen_output)
  return Gen_prob_output

In [18]:
# building Cycle GAN NW

# GAN for approximating A Distribution

X_BA = GenBA(X_B)
Disc_A_real = Disc_A(X_A)
Disc_A_fake = Disc_A(X_BA)

# GAN for approximating B Distribution
X_AB = GenBA(X_A)
Disc_B_real = Disc_A(X_B)
Disc_B_fake = Disc_A(X_AB)


In [19]:
# Discriminator Loss Functions

Loss_Disc_A = (tf.reduce_mean(tf.square(Disc_A_real - tf.ones_like(Disc_A_real)) + tf.reduce_mean(tf.square(Disc_A_fake))))/2
Loss_Disc_B = (tf.reduce_mean(tf.square(Disc_B_real - tf.ones_like(Disc_B_real)) + tf.reduce_mean(tf.square(Disc_B_fake))))/2

Disc_Loss = Loss_Disc_A + Loss_Disc_B

In [20]:
# image reconstruction

X_BAB = GenAB(X_BA)
X_ABA = GenBA(X_AB)

In [21]:
# Generator Loss function
Loss_Gen_A = tf.reduce_mean(tf.square(Disc_B_fake - tf.ones_like(Disc_B_fake)))
Loss_Gen_B = tf.reduce_mean(tf.square(Disc_A_fake - tf.ones_like(Disc_A_fake)))
Loss_total = Loss_Gen_A + Loss_Gen_B

# Reconstruction Loss

Loss_recon_A = tf.reduce_mean(10*tf.abs((X_A-X_ABA)))
Loss_recon_B = tf.reduce_mean(10*tf.abs((X_B-X_BAB)))
Loss_recon_total = Loss_recon_A +Loss_recon_B

Gen_Loss = Loss_total + Loss_recon_total

In [22]:
# parameters list of Discriminator

Disc_param = [Disc_A_W['disc_H'], Disc_A_W["disc_final"], Disc_A_Bias['disc_H'], Disc_A_Bias['disc_final'],
              Disc_B_W['disc_H'], Disc_B_W['disc_final'],Disc_B_Bias['disc_H'], Disc_B_Bias['disc_final'] ]
# parameters list of Gen
Gen_param = [Gen_AB_W['Gen_H'], Gen_AB_W["Gen_final"], Gen_AB_Bias['Gen_H'], Gen_AB_Bias['Gen_final'],
            Gen_BA_W['Gen_H'], Gen_BA_W['Gen_final'],Gen_BA_Bias['Gen_H'], Gen_BA_Bias['Gen_final'] ]


# Define the optimizer

Gen_Optimize = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(Gen_Loss, var_list = Gen_param)
Disc_Optimize = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(Disc_Loss, var_list = Disc_param)

In [23]:
tf.__version__

'2.4.1'

In [24]:
!pip install tensorflow
!pip install -q git+https://github.com/tensorflow/examples.git
!ls

New-Project-25.jpg  New-Project-25.jpg.1  New-Project-25.jpg.2	sample_data


In [26]:
import tensorflow_datasets
mnist = tensorflow_datasets.load('mnist')

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
!mkdir MNIST_Fashion
!cp *.gz MNIST_Fashion/
# import input_data
import tensorflow_datasets
mnist = tensorflow_datasets.load('mnist')

X_train = mnist.train.images
mid = int(X_train.shape[0]/2)

# Real Images Dataset 1
X_train_real = X_train[:mid]

# Rotated images Dataset 2
X_train_rot = X_train[mid:].reshape(-1,28,28)
X_train_rot = scipy.ndimage.interpolation.rotate(X_train_rot, 90, axes = (1,2))
X_train_rot = X_train_rot.reshape(-1,28*28)

# random shuffling

def shuffle_data(x, size):
  start_index = np.random.randint(0, x.shape[0]-size)
  return x[start_index:start_index+size]

In [ ]:
# Initialize the variables

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  X_A_batch = shuffle_data(X_train_real, batch_size)
  X_B_batch = shuffle_data(X_train_rot, batch_size)

  _, Disc_loss_epoch = sess.run([Disc_Optimize, Disc_Loss], feed_dict = {X_A:X_A_batch, X_B:X_B_batch})
  _, Gen_loss_epoch = sess.run([Gen_Optimize, Gen_Loss], feed_dict = {X_A:X_A_batch, X_B:X_B_batch})

  if epoch % 2000 == 0:
    print("steps: {0}, Disc loss; {1}, Gen Loss: {2}".format(epoch, Disc_loss_epoch, Gen_loss_epoch))

In [ ]:
#testing
n = 6
canvas1 = np.empty((28*n, 28*n))
canvas2 = np.empty((28*n, 28*n))

for i in range(n):
  test_A = shuffle_data(X_train_real, batch_size)
  test_B = shuffle_data(X_train_rot, batch_size)

  # Generate A images from B
  out_A = sess.run(X_BA, feed_dict={X_B: test_B})

  # Generate B images from A
  out_B = sess.run(X_AB, feed_dict={X_A: test_A})
  
  for j in range(n):
    # Draw the Generated Digits
    canvas1[i*28: (i + 1) *28, j* 28:(j+1) *28] = out_A[j].reshape([28,28])

  for j in range(n):
    # Draw the Generated Digits
    canvas2[i*28: (i + 1) *28, j* 28:(j+1) *28] = out_B[j].reshape([28,28])

# One way of displaying
plt.figure(figsize=(n, n))
plt.imshow(canvas1, origin = "upper", cmap = "gray")
plt.show()

plt.figure(figsize = (n,n))
plt.imshow(canvas2, origin="upper", cmap="gray")
plt.show()